# Лабораторная работа №8 (Проведение исследований моделями обнаружения и распознавания объектов)

Установка библиотек

In [1]:
!pip install ultralytics kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alekseymarchenko","key":"fa052b6c963aff97173805488e484a51"}'}

Загрузка датасета

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d nimapourmoradi/car-plate-detection-yolov8
!unzip car-plate-detection-yolov8.zip

Dataset URL: https://www.kaggle.com/datasets/nimapourmoradi/car-plate-detection-yolov8
License(s): DbCL-1.0
 79% 142M/180M [00:00<00:00, 481MB/s] 
100% 180M/180M [00:00<00:00, 377MB/s]
Archive:  car-plate-detection-yolov8.zip
  inflating: README.dataset.txt      
  inflating: README.roboflow.txt     
  inflating: Raw_Plate/raw_plate.png  
  inflating: data.yaml               
  inflating: test/images/192_png.rf.3acead9fe79797208877bfbeec07cc26.jpg  
  inflating: test/images/206_png.rf.ed4556ec5eeca1d2f5c82705324285e0.jpg  
  inflating: test/images/209_png.rf.6ebccd08d01c3950fa1fec916fe18a88.jpg  
  inflating: test/images/211_png.rf.3649bb1d1966dbc3cab6288ff17837a0.jpg  
  inflating: test/images/235_png.rf.b15a717d02bf70e4dcfc467b582751b6.jpg  
  inflating: test/images/250_png.rf.5a524c9c26d597a867a74258dd957f2f.jpg  
  inflating: test/images/280_png.rf.80aed7b81ae217f30e00d63972764ac7.jpg  
  inflating: test/images/288_png.rf.8e37dba6dcb939f6cd2565d5340b4934.jpg  
  inflating: test/ima

Обучение модели

In [ ]:
!yolo train data=data.yaml model=yolov8s.pt epochs=10 imgsz=640 device=0

Визуализация изображений

In [ ]:
from IPython.display import display, Image
import os
import glob

predict_folders = glob.glob('runs/detect/predict*/')
latest_folder = max(predict_folders, key=os.path.getmtime)
pred_images = glob.glob(os.path.join(latest_folder, '*.jpg'))

# Первые 5 изображений
for img_path in pred_images[:5]:
    print(f"Отображаю: {os.path.basename(img_path)}")
    display(Image(filename=img_path, width=600))

## Улучшенный Бейзлайн

Добавление слоёв в голову модели

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8m.pt')
model.model.yaml['head'] = ['Conv', 'Detect']  # Пример модификации
model.train(data='data.yaml', epochs=50, imgsz=640)

Обучение с измененными гиперпараметрами

In [ ]:
!yolo train data=data.yaml model=yolov8m.pt epochs=50 imgsz=640 batch=16 lr0=0.01 weight_decay=0.0005 device=0
!yolo val data=data.yaml model=runs/train/weights/best.pt device=0

Оценка качества моделей с улучшенным бейзлайном

In [ ]:
from ultralytics import YOLO
model = YOLO('runs/train/weights/best.pt')
metrics = model.val()
print(f"mAP@0.5: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")
print(f"IoU: {metrics.box.miou:.4f}")

Сравнение результатов с исходным бейзлайном

In [ ]:
model_s = YOLO('runs/train_s/weights/best.pt')
metrics_s = model_s.val()

# Сравнение
print("Сравнение метрик:")
print(f"{'Метрика':<10} | {'YOLOv8s':<8} | {'YOLOv8m (улучшенный)'}")
print("-"*40)
print(f"{'mAP@0.5':<10} | {metrics_s.box.map:.4f} | {metrics.box.map:.4f}")
print(f"{'Precision':<10} | {metrics_s.box.mp:.4f} | {metrics.box.mp:.4f}")
print(f"{'Recall':<10} | {metrics_s.box.mr:.4f} | {metrics.box.mr:.4f}")
print(f"{'IoU':<10} | {metrics_s.box.miou:.4f} | {metrics.box.miou:.4f}")

# Имплементация алгоритма машинного обучения

Реализация модели

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

class MyCNN(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()
        self.backbone = nn.Sequential(
            nn.Conv2d(3, 16, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 14 * 14, 512),
            nn.ReLU(),
            nn.Linear(512, 4)  # [x_center, y_center, width, height]
        )

    def forward(self, x):
        x = self.backbone(x)
        return self.head(x)

Обучение модели

In [ ]:
class CarPlateDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, img) for img in os.listdir(root_dir)]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB').resize((224, 224))
        if self.transform:
            image = self.transform(image)
        bbox = torch.tensor([0.5, 0.5, 0.2, 0.1], dtype=torch.float32)
        return image, bbox

# Преобразования
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Датасет и загрузчик
train_dataset = CarPlateDataset('train/images', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Обучение
model = MyCNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    for images, bboxes in train_loader:
        outputs = model(images)
        loss = criterion(outputs, bboxes)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Оценка качества имплементированной модели

In [ ]:
import torch
from torchvision.ops import box_iou

# Функция вычисления IoU
def compute_iou(pred_boxes, true_boxes):
    return box_iou(pred_boxes, true_boxes)

# Пример вычисления метрик для MyCNN
model.eval()
total_iou = 0
total_precision = 0
total_recall = 0
num_samples = 0

with torch.no_grad():
    for images, true_boxes in test_loader:
        pred_boxes = model(images)
        iou = compute_iou(pred_boxes, true_boxes)
        total_iou += iou.diag().sum().item()
        # Precision и Recall (пример простого порога IoU > 0.5)
        correct = (iou.diag() > 0.5).sum().item()
        total_precision += correct / pred_boxes.shape[0]
        total_recall += correct / true_boxes.shape[0]
        num_samples += images.shape[0]

avg_iou = total_iou / num_samples
avg_precision = total_precision / num_samples
avg_recall = total_recall / num_samples

print(f"IoU: {avg_iou:.4f}")
print(f"Precision: {avg_precision:.4f}")
print(f"Recall: {avg_recall:.4f}")

Добавление техник из улучшенного бейзлайна

In [ ]:
# Добавление аугментаций
transform = transforms.Compose([
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Обучение с аугментациями
train_dataset = CarPlateDataset('train/images', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Обучение модели с аугментациями
model = MyCNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    for images, bboxes in train_loader:
        outputs = model(images)
        loss = criterion(outputs, bboxes)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Оценка качества моделей с техниками из улучшенного бейзлайна

In [ ]:
model.eval()
total_iou_aug = 0
total_precision_aug = 0
total_recall_aug = 0
num_samples_aug = 0

with torch.no_grad():
    for images, true_boxes in test_loader:
        pred_boxes = model(images)
        iou = compute_iou(pred_boxes, true_boxes)
        total_iou_aug += iou.diag().sum().item()
        correct = (iou.diag() > 0.5).sum().item()
        total_precision_aug += correct / pred_boxes.shape[0]
        total_recall_aug += correct / true_boxes.shape[0]
        num_samples_aug += images.shape[0]

avg_iou_aug = total_iou_aug / num_samples_aug
avg_precision_aug = total_precision_aug / num_samples_aug
avg_recall_aug = total_recall_aug / num_samples_aug

print(f"IoU (с аугментациями): {avg_iou_aug:.4f}")
print(f"Precision (с аугментациями): {avg_precision_aug:.4f}")
print(f"Recall (с аугментациями): {avg_recall_aug:.4f}")

Сравнение SimpleCNN и YOLOv8m

In [ ]:
print("Сравнение SimpleCNN и YOLOv8m:")
print(f"{'Метрика':<10} | {'SimpleCNN':<10} | {'YOLOv8m'}")
print("-"*40)
print(f"{'IoU':<10} | {avg_iou_aug:.4f} | {metrics.box.miou:.4f}")
print(f"{'Precision':<10} | {avg_precision_aug:.4f} | {metrics.box.mp:.4f}")
print(f"{'Recall':<10} | {avg_recall_aug:.4f} | {metrics.box.mr:.4f}")

Визуализация результатов

In [ ]:
import matplotlib.pyplot as plt

# График сравнения метрик
metrics_names = ['IoU', 'Precision', 'Recall']
cnn_values = [avg_iou_aug, avg_precision_aug, avg_recall_aug]
yolo_values = [metrics.box.miou, metrics.box.mp, metrics.box.mr]

x = range(len(metrics_names))

plt.figure(figsize=(10, 5))
plt.bar(x, cnn_values, width=0.4, label='SimpleCNN', color='skyblue')
plt.bar([i + 0.4 for i in x], yolo_values, width=0.4, label='YOLOv8m', color='salmon')
plt.xticks([i + 0.2 for i in x], metrics_names)
plt.ylabel('Значение')
plt.title('Сравнение метрик')
plt.legend()
plt.show()